## Parse

In [ ]:
from __future__ import annotations

import re
import os
import json
import dataclasses
import matplotlib.pyplot as plt

from typing import Tuple
from collections import defaultdict as dd


@dataclasses.dataclass(frozen=True)
class ExperimentConfig:
    paxos_variant: str
    number_clients: int
    number_replicas: int
    payload: int

    @staticmethod
    def from_obj(obj: dict) -> ExperimentConfig:
        return ExperimentConfig(
            obj["paxos_variant"],
            obj["number_clients"],
            obj["number_replicas"],
            obj["payload"],
        )


@dataclasses.dataclass(frozen=True)
class ExperimentResults:
    throughput: float
    read_latency_ms: float
    write_latency_ms: float
    overall_latency_ms: float

    @staticmethod
    def parse_from_output(output: str) -> ExperimentResults:
        read_latency_ms = None
        write_latency_ms = None
        throughput = None
        lines = output.split("\n")
        for line in lines:
            if "[OVERALL], Throughput(ops/sec), " in line:
                throughput = float(re.findall(r"\d+\.\d+", line)[0])
            elif "[READ], AverageLatency(us), " in line:
                read_latency_ms = float(re.findall(r"\d+\.\d+", line)[0]) / 1000
            elif "[UPDATE], AverageLatency(us), " in line:
                write_latency_ms = float(re.findall(r"\d+\.\d+", line)[0]) / 1000
            elif "Op Timed out" in line:
                return None

        assert read_latency_ms is not None
        assert write_latency_ms is not None
        assert throughput is not None

        overall_latency_ms = (read_latency_ms + write_latency_ms) / 2
        return ExperimentResults(
            throughput, read_latency_ms, write_latency_ms, overall_latency_ms
        )


@dataclasses.dataclass(frozen=True)
class ExperimentOutput:
    config: ExperimentConfig
    results: ExperimentResults

    @staticmethod
    def from_file(contents: str) -> ExperimentOutput:
        obj = json.loads(contents)
        config = ExperimentConfig.from_obj(obj["config"])
        results = ExperimentResults.parse_from_output(obj["output"])
        return ExperimentOutput(config, results)

In [ ]:
# Read in all experiment results

EXPERIMENTS_DIR = "experiments"
EXPERIMENTS: list[ExperimentOutput] = []

for result_file in os.listdir(EXPERIMENTS_DIR):
    with open(os.path.join(EXPERIMENTS_DIR, result_file), "r") as f:
        contents = f.read()
        EXPERIMENTS.append(ExperimentOutput.from_file(contents))

print(f"Loaded {len(EXPERIMENTS)} experiments")

## Plot

In [ ]:
GRAPHS_FOLDER = "graphs/"
os.makedirs(GRAPHS_FOLDER, exist_ok=True)


@dataclasses.dataclass(frozen=True)
class Line:
    variant: str
    replicas: int
    payload: int
    throughput_axis: list[float]
    latency_axis: list[float]


@dataclasses.dataclass(frozen=True)
class LineGroup:
    replicas: int
    payload: int
    lines: list[Line]


def pretty_paxos_variant(paxos_variant: str) -> str:
    if paxos_variant == "single":
        return "Paxos"
    elif paxos_variant == "multi":
        return "Multi-Paxos"
    else:
        return paxos_variant


def gen_lines() -> list[Line]:
    group_keys = set(
        map(
            lambda x: (
                x.config.paxos_variant,
                x.config.number_replicas,
                x.config.payload,
            ),
            EXPERIMENTS,
        )
    )
    group_exps = {
        group_key: [
            exp
            for exp in EXPERIMENTS
            if (
                exp.config.paxos_variant,
                exp.config.number_replicas,
                exp.config.payload,
            )
            == group_key
        ]
        for group_key in group_keys
    }

    lines = []
    for group_key, exps in group_exps.items():
        variant, replicas, payload = group_key
        troughput_axis = []
        latency_axis = []
        for exp in sorted(exps, key=lambda x: x.config.number_clients):
            troughput_axis.append(exp.results.throughput / 1000)
            latency_axis.append(exp.results.overall_latency_ms)
        lines.append(
            Line(
                variant=variant,
                replicas=replicas,
                payload=payload,
                throughput_axis=troughput_axis,
                latency_axis=latency_axis,
            )
        )
    return lines


def gen_groups() -> list[LineGroup]:
    group_lines = dd(list)
    for line in gen_lines():
        key = (line.replicas, line.payload)
        lines = group_lines[key]
        lines.append(line)

    return [
        LineGroup(replicas, payload, lines)
        for (replicas, payload), lines in group_lines.items()
    ]


def gen_throughput_latency_graph(group: LineGroup):
    for line in group.lines:
        plt.plot(
            line.throughput_axis,
            line.latency_axis,
            marker="*",
            label=pretty_paxos_variant(line.variant),
        )
    plt.xlabel("Throughput (1000 ops/sec)")
    plt.ylabel("Average Latency (ms)")
    plt.title(f"replicas={group.replicas}, payload={group.payload}B")

    plt.tight_layout()
    plt.legend()

    plt.savefig(f"{GRAPHS_FOLDER}{group.replicas}R_{group.payload}B.pdf")
    plt.show()
    plt.clf()


for group in gen_groups():
    gen_throughput_latency_graph(group)